<a href="https://colab.research.google.com/github/nsk-ai/RAG-Bootcamp-2025/blob/main/Sparse_Vector_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sparse Vector Search

In [1]:
pip install langchain

In [2]:
pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 6.1 MB/s eta 0:00:00


In [4]:
from google.colab import userdata
import os

# Access the secret using userdata.get()
my_variable = userdata.get('GROQ_API_KEY')

# You can also set it as an environment variable for use with os.getenv()
os.environ['GROQ_API_KEY'] = my_variable

In [5]:
%pip install -qU langchain_community rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.1 MB/s eta 0:00:00


In [13]:
# --- Setup: Imagine these are your chunks ---
from langchain_core.documents import Document

sample_chunks = [
    Document(page_content="The cat sat on the mat. It was a fluffy cat.", metadata={"source": "story-1", "page": 1}),
    Document(page_content="The dog chased the ball. The ball was red.", metadata={"source": "story-2", "page": 1}),
    Document(page_content="A fluffy feline rested on the soft rug.", metadata={"source": "story-1", "page": 2}),
    Document(page_content="Dogs are known for their loyalty and playful nature.", metadata={"source": "story-2", "page": 2}),
    Document(page_content="The new project report for Q3 is due next week.", metadata={"source": "report-q3"}),
    Document(page_content="Q4 financial results show significant growth.", metadata={"source": "report-q4"}),
]

In [14]:
# --- BM25 Implementation ---
from langchain_community.retrievers import BM25Retriever

# 1. Initialize BM25Retriever from our documents
bm25_retriever = BM25Retriever.from_documents(sample_chunks)
bm25_retriever.k = 2 # We want the top 2 results

In [15]:
# Test 1: A query with strong keyword overlap
query1 = "fluffy cat"
print(f"\nQuery: '{query1}'")
retrieved_docs_1 = bm25_retriever.invoke(query1)
for i, doc in enumerate(retrieved_docs_1):
    print(f"Doc {i+1} (Source: {doc.metadata.get('source')}): {doc.page_content}")


Query: 'fluffy cat'
Doc 1 (Source: story-1): The cat sat on the mat. It was a fluffy cat.
Doc 2 (Source: story-1): A fluffy feline rested on the soft rug.


In [23]:
# Test 2: A query with a specific, rare term
query2 = "Q3 report"
print(f"\nQuery: '{query2}'")
retrieved_docs_2 = bm25_retriever.invoke(query2)
for i, doc in enumerate(retrieved_docs_2):
    print(f"Doc {i+1} (Source: {doc.metadata.get('source')}): {doc.page_content}")


Query: 'Q3 report'
Doc 1 (Source: report-q3): The new project report for Q3 is due next week.
Doc 2 (Source: report-q4): Q4 financial results show significant growth.


In [22]:
# Test 3: A semantic query where keywords DON'T match (demonstrating the weakness)
query3 = "What do you think about felines"
print(f"\nQuery: '{query3}'")
retrieved_docs_3 = bm25_retriever.invoke(query3)
for i, doc in enumerate(retrieved_docs_3):
    print(f"Doc {i+1} (Source: {doc.metadata.get('source')}): {doc.page_content}")



Query: 'What do you think about felines'
Doc 1 (Source: report-q4): Q4 financial results show significant growth.
Doc 2 (Source: report-q3): The new project report for Q3 is due next week.
